In [1]:
import numpy as np
import gurobipy as gp


# OJ Example - Sensitivity Analysis

In [2]:
obj = np.array([500,450]) # objective vector
A = np.zeros((3,2)) # initialize constraint matrix
A[0,:] = [6,5] # production constraint
A[1,:] = [10,20] # storage constraint
A[2,:] = [1,0] # demand constraint
b = np.array([60,150,8]) # limits on production, storage, and demand
sense = np.array(['<','<','<']) # all constraints are less than or equal constraints

In [3]:
ojModel = gp.Model() # initialize an empty model

ojMod_x = ojModel.addMVar(2) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
ojMod_con = ojModel.addMConstrs(A, ojMod_x, sense, b) # NAME THE CONSTRAINTS!!! so we can get information about them later!
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

ojModel.Params.OutputFlag = 0 # tell gurobi to shut up!!
ojModel.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14


In [4]:
ojModel.Objval

5142.857142857143

In [5]:
ojMod_x.x

array([6.42857143, 4.28571429])

# Sensitivity Analysis

objMod_con is a list of gurobi objects.  

Each constraint is an item in the list of objects.

Each item/object has several attributes and methods.

If we want to access sensitivity analysis for each constraint we must iterate through the constraint list of objects and extract the relevant attributes.




# Shadow Costs / Dual Variables

In [6]:
[ojMod_con[0].Pi , ojMod_con[1].Pi , ojMod_con[2].Pi] #each array is the incrememnt in amount of money we make for increment of each variable

[78.57142857142857, 2.857142857142857, 0.0]

In [7]:
[con.Pi for con in ojMod_con]

[78.57142857142857, 2.857142857142857, 0.0]

# Upper and lower bound where duals are valid

In [8]:
[con.SARHSLow for con in ojMod_con]

[37.5, 128.0, 6.428571428571429]

In [9]:
[con.SARHSUp for con in ojMod_con]

[65.5, 240.0, inf]

## Double check objective with shadow price

In [10]:
db = 10
ojModel.Objval + db*ojMod_con[0].Pi

5928.571428571428

In [11]:
b2 = np.array([60+db,150,8])
ojModel2 = gp.Model() # initialize an empty model

ojMod_x2 = ojModel2.addMVar(2) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
ojMod_con2 = ojModel2.addMConstrs(A, ojMod_x2, sense, b2) # NAME THE CONSTRAINTS!!! so we can get information about them later!
ojModel2.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

ojModel2.Params.OutputFlag = 0 # tell gurobi to shut up!!
ojModel2.optimize()

ojModel2.Objval

5575.0

# Range of obj where optimal corner stays the same

In [12]:
ojMod_x.SAObjLow

array([225.        , 416.66666667])

In [13]:
ojMod_x.SAObjUp

array([ 540., 1000.])